# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import os

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [4]:
# get datasets for white resumes and black resumes
w = data[data.race=='w']
b = data[data.race=='b']

# number of callbacks for black-sounding names
callback_w = sum(data[data.race=='w'].call)
callback_b = sum(data[data.race=='b'].call)

# total black and white assigned resumes
count_w = len(data[data.race=='w'])
count_b = len(data[data.race=='b'])

# rate of callback for white and black sounding names
rate_w = callback_w/count_w
rate_b = callback_b/count_b
rate_diff = rate_w-rate_b

print('w sum', callback_w, count_w)
print('b sum', callback_b, count_b)

w sum 235.0 2435
b sum 157.0 2435


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

1. The difference in proportions is the test statistic.  The Z-test statistic (for the difference between proportions) should be used.  Since the datasets have binomial distributions, the p-value can be approximated using the normal distribution.  The CLT does apply in this case.

2. 
H0: Race does not have a significant impact on the rate of callbacks for resumes.  The difference in callback rate for white sounding names versus black sounding names is <= 0 (w-b <= 0) <br>
H1: Race has a significant impact on the rate of callbacks for resumes.  The difference in callback rate for white sounding names versus black sounding names is > 0 (w-b > 0)


<div class="span5 alert alert-success">
<p>Solution to Q3</p>
</div>

In [5]:
#confidence interval and p-value using permutation:

In [6]:
from sklearn.utils import resample

In [45]:
def permutation_sample(data1, data2, size=10):
    """Generate a permutation sample from two data sets."""
    
    perm_replicates = np.empty(size)
    
    for i in range(size):
        # Generate permutation sample
        data = np.concatenate((data1.call,data2.call))
        # Permute the concatenated array: permuted_data
        permuted_data = np.random.permutation(data)

        # Split the permuted array into two: perm_sample_1, perm_sample_2
        perm_sample_1 = permuted_data[:len(data1)]
        perm_sample_2 = permuted_data[len(data1):]

        p1 = perm_sample_1.sum()/len(data1)
        p2 = perm_sample_2.sum()/len(data2)
        diff = p1-p2
        
        perm_replicates[i] = diff

    return perm_replicates


def bootstrap_confidence_interval(data1, data2, size=10000, alpha=.05):
    """Generate a confidence interval based on bootstrap samples of each individual dataset, return the 
    lower and upper bound of a 1-alpha% confidence interval"""

    bs_replicates = np.empty(size)
    
    for i in range(size):
        # resample each of the datasets
        data1_resample, data2_resample = resample(data1, data2)
        p1 = data1_resample.call.sum()/len(data1)
        p2 = data2_resample.call.sum()/len(data2)
        # compute difference in resampled proportions
        diff = p1-p2
        
        bs_replicates[i] = diff
    
    # compute the confidence interval using the percentile method
    lower=alpha/2*100
    upper=100-alpha/2*100
    conf_int = np.percentile(bs_replicates, [lower,upper])
    
    return conf_int
        

def bootstrap_sample(data1, data2, size=10000):
    """Generate bootstrap replicates based on bootstrap resamples for combined dataset, return array
    of difference in proportions, based on the resampled data"""
    bs_replicates = np.empty(size)
    
    # combine the data
    data = np.concatenate((data1.call,data2.call))
    
    for i in range(size):
        # using sklearn resample method for convenience, could also use np.random.choice
        data_resample = resample(data)

        p1 = data_resample[:len(data1)].sum()/len(data1)
        p2 = data_resample[len(data1):].sum()/len(data2)
        diff = p1-p2
        
        bs_replicates[i] = diff
        
    return bs_replicates


In [31]:
# if the callback rate for black sounding names and white sounding names were identical, the probability of observing
# a difference in rates as high as the observed difference would be .00003.
size = 100000
reps_perm = permutation_sample(w, b, size=size)
p_value = np.sum(reps_perm >= rate_diff) / size
print(p_value)

3e-05


In [18]:
# and if the callback rates were identical, the 95% confidence interval would narrowly surround 0.
conf_int = np.percentile(reps_perm, [2.5, 97.5])
conf_int

array([-0.01560575,  0.01560575])

In [43]:
# using a bootstrap (rather than permutation) resampling, the pvalue would be .00002
reps_bs = bootstrap_sample(w, b, size=size)
p_value = np.sum(reps_bs2 >= rate_diff) / size
print(p_value)

2e-05


In [25]:
# again, assuming the callback rates were identical, the 95% confidence interval using bootstrap replicates would 
# narrowly surround 0
conf_int = np.percentile(reps_bs2, [2.5, 97.5])
conf_int

array([-0.01519507,  0.01519507])

In [46]:
# the confidence interval based on boostrap samples of each original w vs b dataset does not include 0:
conf = bootstrap_confidence_interval(w, b, alpha=.05)
print(conf)

[0.01971253 0.04435318]


In [47]:
# calculate the z-statistic for the difference between proportions
import math
from scipy.stats import norm

def z_stat_proportions(data1, data2):
    """Calculate z-test statistic for the difference between two proportions"""

    # total callbacks for each dataset
    sum1 = sum(data1.call)
    sum2 = sum(data2.call)
    
    # number of observations/applications for each dataset
    count1 = len(data1)
    count2 = len(data2)
    
    # proportion of callbacks per application for each dataset
    p1 = sum1/count1
    p2 = sum2/count2   

    # combined proportion
    p = (sum1+sum2)/(count1+count2)
    
    # standard error
    se = p*(1-p)*(1/count1+1/count2)
    se = math.sqrt(se)
    
    z = (p1-p2)/se
                       
    return z


def get_pval(z_stat, two_sided=True):
    """Get the p-value for the given z-statistic"""
    pval = norm.sf(abs(z_stat))
    
    if two_sided:
        pval = pval*2

    return pval

In [48]:
ztest = z_stat_proportions(w,b)
pval = get_pval(ztest)
print(pval)

3.983886837585077e-05


In [49]:
# margin of error: 

def margin_of_error_proportions(alpha, p1, n1, p2, n2):
    """Calculate the margin of error based on alpha and given p1 and p2 proportions and 
    n1 and n2 number of observations for two datasets"""
    z_alpha = norm.isf(alpha/2)
    me = z_alpha * math.sqrt((p1*(1-p1)/n1) + (p2*(1-p2)/n2))
    
    return me

In [50]:
# confidence interval:

def confidence_int(margin_of_error, p1, p2):
    lower = p1-p2-margin_of_error
    upper = p1-p2+margin_of_error
    
    return (lower, upper)

margin_of_error = margin_of_error_proportions(.05, rate_w, count_w, rate_b, count_b)
confidence_interval = confidence_int(margin_of_error, rate_w, rate_b)
print(confidence_interval)

(0.01677772818123075, 0.04728798023766042)


<div class="span5 alert alert-success">
<p>Your answers to Q4 and Q5 here</p>
</div>

#4. The analysis above shows that there is a statistically signficant difference in callback rates based on an applicant's race, which indicates race is an important factor in the labor market.  The difference in callback rates for white sounding names versus black sounding names was 3.2% with a 95% confidence interval of 1.7%-4.7%.  

#5. While the analysis shows race is an important factor in the labor market, it does not necessarily mean that it is the most important factor.  In this study, all other factors were equal between the two datasets (black sounding versus white sounding names).  To determine the effect of other factors, these other factors should be subjected to the same treatment and control methods (random assignment of the factor between two groups, keep all other factors equal).